In [1]:

import graph_definition
from langchain_core.messages import HumanMessage
import json
from tqdm import tqdm
import utils.my_langchain_tools as my_langchain_tools 

2025-01-08 21:22:40.758 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-08 21:22:40.759 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`


---my_langchain_tools---, st.session_state {}


In [2]:
qa_set = json.load(open('evaluation/qa_set.json'))

In [3]:
qa_set

[{'question': 'What is the MOA of hydroflumethiazide?',
  'query': "FROM GRAPH_TABLE (drug_graph\n    MATCH\n        (d:Drug WHERE LOWER(d.name) = LOWER('hydroflumethiazide'))-[h:HAS_MOA]->(m:MOA)\n COLUMNS (m.name AS moa_name)\n    ) LIMIT 30;",
  'expected_answer': 'Sodium Chloride Symporter Inhibitors',
  'draft_query': "FROM GRAPH_TABLE (drug_graph\n    MATCH\n        (d:Drug WHERE LOWER(d.name) = LOWER('hydroflumethiazide'))-[h:HAS_MOA]->(m:MOA)\n COLUMNS (m.name AS moa_name)) LIMIT 5;",
  'generated_answer': 'The mechanism of action (MOA) of hydroflumethiazide is categorized as follows: Inhibitor Type: Sodium Chloride Symporter Inhibitors. This indicates that hydroflumethiazide works by inhibiting the sodium chloride symporter.'},
 {'question': 'What diseases can hydroflumethiazide treat?',
  'query': "FROM GRAPH_TABLE (drug_graph  \n    MATCH  \n        (i:Drug WHERE LOWER(i.name) = LOWER('hydroflumethiazide'))-[m:MAY_TREAT]->(c:Disorder)  \n    COLUMNS (c.name AS disorder_name)

In [4]:
config = {"configurable": {"thread_id": "1", "user_id": "sixing"}}

new_set = []

for qa in tqdm(qa_set):
  generated_answer = ""
  retry = 0
  while retry < 3:
    try:
      input_message = HumanMessage(
          content=my_langchain_tools.expand_question(qa["question"]), tool_choice="automatic"
          #content="""Calculate the amount of trials sponsored by each of "the Three small guys".""", tool_choice="sql"
      )

      for event in graph_definition.app.stream({"messages": [input_message]}, config, stream_mode="values"):
          #print ("len:", len(event["messages"]))
          pass

      #print (graph_definition.app.get_state(config).values["messages"][-1])

      tool_call_id = graph_definition.app.get_state(config).values["messages"][-1].tool_call_id
      tool_name = graph_definition.app.get_state(config).values["messages"][-1].name
      tool_content = graph_definition.app.get_state(config).values["messages"][-1].content

      # We now create the tool call with the id and the response we want
      tool_message = [
          {
          "tool_call_id": tool_call_id, 
          "name": tool_name, 
          #"type": "tool",
          "type": "human",
              "content": tool_content}
      ]

      # # This is equivalent to the below, either one works
      # from langchain_core.messages import ToolMessage
      # tool_message = [ToolMessage(tool_call_id=tool_call_id, content="san francisco")]

      # We now update the state
      # Notice that we are also specifying `as_node="ask_human"`
      # This will apply this update as this node,
      # which will make it so that afterwards it continues as normal
      graph_definition.app.update_state(config, {"messages": tool_message}, as_node="human_feedback")
      events = list(graph_definition.app.stream(None, config, stream_mode="values"))
      last_event = events[-1]
      generated_answer = last_event["messages"][-1].content
      break
    except Exception as e:
      print ("ERROR!!!:", e)
      retry += 1

  new_set.append({"question": qa["question"], "generated_answer": generated_answer, "expected_answer": qa["expected_answer"]})

  0%|          | 0/16 [00:00<?, ?it/s]

---expand_question---
 question: What is the MOA of hydroflumethiazide?
*****term_extractor question What is the MOA of hydroflumethiazide?
function_args {'drugs': ['hydroflumethiazide'], 'disorders': [], 'mechanisms': []}
---expand_question---
 terms: {'hydroflumethiazide'}
Before response
---select_intent---
response:  content='' additional_kwargs={'tool_calls': [{'id': 'call_KhanM1HxVnN19SYMuWbf1oJJ', 'function': {'arguments': '{"action_type":"limit_query_tool"}', 'name': 'Choose_Direction'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 317, 'total_tokens': 336, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_f3927aa00d', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hat

  6%|▋         | 1/16 [00:06<01:35,  6.36s/it]

return tool_call_id call_044lNFtALs9kxG8307uPIwX0
---expand_question---
 question: What diseases can hydroflumethiazide treat?
*****term_extractor question What diseases can hydroflumethiazide treat?
function_args {'drugs': ['hydroflumethiazide'], 'disorders': [], 'mechanisms': []}
---expand_question---
 terms: {'hydroflumethiazide'}
Before response
---select_intent---
response:  content='' additional_kwargs={'tool_calls': [{'id': 'call_XrwRWAGRYH7qEhK0LeZXp0Ld', 'function': {'arguments': '{"action_type":"limit_query_tool"}', 'name': 'Choose_Direction'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 592, 'total_tokens': 611, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_f3927aa00d', 'prompt_filter_re

 12%|█▎        | 2/16 [00:12<01:29,  6.43s/it]

...

write_query first=RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
}) middle=[RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')}), FewShotPromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain_community.vectorstores.lancedb.LanceDB object at 0x16a1d8c90>, k=5, example_keys=None, input_keys=['input'], vectorstore_kwargs=None), example_prompt=PromptTemplate(input_variables=['input', 'query'], input_types={}, partial_variables={}, template='User input: {input}\nSQL query: {query}'), suffix='User input: {input}\nSQL query: ', prefix="You are a DuckDB expert. Given an input question, create a syntactically correct DuckDB query to run. Ignore the {top_k} parameter for now.\n        Here is the relevant table info: {table_info}\n        In the question, if you see the '(CUI: xxx)' i

 94%|█████████▍| 15/16 [01:54<00:08,  8.12s/it]

return tool_call_id call_rexlNRULVjkzLaOz2W4QgmEh
---expand_question---
 question: Which substance that may treat Type 1 Diabetes but does not have the MOA Insulin Receptor Agonists
*****term_extractor question Which substance that may treat Type 1 Diabetes but does not have the MOA Insulin Receptor Agonists
function_args {'drugs': [], 'disorders': ['Type 1 Diabetes'], 'mechanisms': ['Insulin Receptor Agonists']}
---expand_question---
 terms: {'Type 1 Diabetes', 'Insulin Receptor Agonists'}
Before response
---select_intent---
response:  content='' additional_kwargs={'tool_calls': [{'id': 'call_MmuhN550cpINubzgcuECoRE3', 'function': {'arguments': '{"action_type":"limit_query_tool"}', 'name': 'Choose_Direction'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 5830, 'total_tokens': 5849, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 

100%|██████████| 16/16 [02:15<00:00,  8.49s/it]

query SELECT i.name
FROM GRAPH_TABLE(
    drug_graph 
    MATCH (i:Drug)-[m:MAY_TREAT]->(c:Disorder WHERE c.disorder_cui = 'C0011854'),
          (i:Drug)-[n:HAS_MOA]->(a:MOA)
    WHERE a.moa_id <> 'C1373092'
    COLUMNS (i.name)
)
LIMIT 30;
---execute_query_and_answer---
---execute_query_and_answer---   m content='' additional_kwargs={'tool_calls': [{'id': 'call_vZfrKaGN55Gx1KcB5huNAx6x', 'function': {'arguments': '{"question":"Which substance that may treat Diabetes Mellitus, Insulin-Dependent (CUI: C0011854) but does not have the MOA Insulin Receptor Agonists (CUI: C1373092)","top_k":30}', 'name': 'mimicking'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 1341, 'total_tokens': 1405, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-0

In [5]:
with open('evaluation/drugbot_data.json', 'w') as f:
    json.dump(new_set, f, indent=2)